In [5]:
from sklearn.metrics import classification_report, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd


%store -r export_df
df = export_df
print(df)

x = df.drop(columns=['activity'])
y = df['activity']


x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=42, stratify=y
)

forest = RandomForestClassifier(n_estimators=200, random_state=42)

forest.fit(x_train, y_train)

predictions = forest.predict(x_test)

acc = classification_report(y_test, predictions)

print(acc)
print(df['activity'].value_counts())


           open       high        low      close        dxy      rsi_6  \
49    115003.76  115110.00  115003.76  115109.99  97.300003  62.042073   
50    115110.00  115120.00  115048.08  115070.00  97.300003  56.817640   
51    115070.01  115074.51  114980.00  114991.99  97.300003  47.461901   
52    114991.99  114991.99  114880.00  114942.19  97.300003  42.145623   
53    114942.19  114942.20  114849.99  114885.24  97.300003  36.530449   
...         ...        ...        ...        ...        ...        ...   
4313  113885.01  113955.91  113870.00  113921.45  97.773003  48.328622   
4314  113921.44  113921.45  113889.89  113889.89  97.773003  41.318937   
4315  113889.89  113926.43  113869.84  113911.50  97.773003  47.567674   
4316  113911.49  113994.61  113911.49  113952.52  97.773003  57.802929   
4317  113952.52  113952.52  113815.85  113815.86  97.773003  32.465955   

         rsi_12       macd  macd_signal  macd_hist         ema_21  \
49    57.750950  15.702564   -25.224870  4